In [1]:
#write instructions
from tkinter import *
import matplotlib
matplotlib.use('TkAgg')
#other gui mode im trying
#%matplotlib widget
#calling it a second time may prevent some graphics errors
#%matplotlib widget  
from matplotlib import pyplot as plt
import pandas as pd
import pyproj
import numpy as np
from datetime import datetime
import utm
from pygeodesy import geoids
import glob
import os
sys.path.append('/home/spike/repos')
from getdatatestbed.getDataFRF import getObs
import math
import pickle
import crawlerFunctions as cf


In [2]:
data = pd.read_pickle("./forMatthew.pickle")

In [24]:
plt.figure(figsize=(12,8))
plt.scatter(data["xFRF"], data["yFRF"], c=data["time"], cmap='plasma', s=2)
#plt.scatter(data["xFRF"], data["yFRF"], c=data["time"], ".", markersize=1, cmap="plasma")
plt.title('please click (left mouse button) start/end points of profilee line\nclick middle button DO NOT CLOSE the window when you''re done')
plt.xlabel("FRF Coordinate System X (m)")
plt.ylabel("FRF Coordinate System Y (m)")
plt.tight_layout()
nodes = plt.ginput(-1, 0)
print(nodes)

[(95.54372615255376, 123.77233718355484), (262.0154562041322, 134.20244400914962), (258.1769090306128, 193.9385103739198), (98.77618693025431, 183.508403548325), (96.3518413469789, 154.1144661307397), (258.5809666278254, 168.33733907473257), (259.18705302364424, 233.7625546170999), (98.37212933304174, 228.07340543950275), (93.92749576370349, 192.99031884432026), (257.1667650375814, 206.26500025871363), (255.55053464873112, 273.58659886028), (90.49300618739665, 264.10468356428476), (100.3924173191046, 246.08904450189374), (261.0053122111008, 268.8456412122824), (256.56067864176254, 290.65404639307144), (139.7880330473301, 281.1721310970762), (114.33240442293823, 353.2346873466402), (269.49052175256475, 349.4419212282421), (269.49052175256475, 349.4419212282421), (270.50066574559617, 354.1828788762397), (109.28168445778111, 375.99128405702885), (267.67226256510816, 372.19851793863074), (267.2682049678956, 386.4213908826236), (134.13122668635413, 393.0587315898203), (110.89791484663138, 4

In [26]:
pickle.dump(nodes, open('coordsDemo.pkl', 'wb'))  # @matt, check this out. way cleaner
#     nodes.dump(nodes, f)

In [27]:
with open('coordsDemo.pkl', 'rb') as f:
    nodes = pickle.load(f)

In [28]:
#match user selected endpoints to crawler data points
# @matthew look in testbedutils.sblib.closestRadialNode will return you the index of what is the closest point 
# so you don't have to do all of this 
#idx = closestRadialNode([xOfInterest, yOfInterest], [xpoints2Search, yPoints2Search])
#endpts.append(data['xfrf'].iloc[idx], data['yFRF'].iloc[idx])
# this is also slooooow it seams 
endpts = []
for x in range(len(nodes)):
    curr = nodes[x]
    prevDist = float('inf')
    closest = tuple()
    for y in range(data.shape[0]):
        dist = math.sqrt((data["yFRF"][y]-curr[1])**2 + (data["xFRF"][y]-curr[0])**2)
        if dist < prevDist:
            prevDist = dist
            closest = (data["xFRF"][y], data["yFRF"][y])
    endpts.append(closest)

In [29]:
#identify endpoints within data frame
#@matt is it worth separating start/end points? i'm guessing here is the place?
isEndPt = []
for x in range(data.shape[0]):
    if (data["xFRF"][x], data["yFRF"][x]) in endpts:
        isEndPt.append(True)
    else:
        isEndPt.append(False)
data["endPt"] = isEndPt

In [30]:
#identify and number transects within dataframe
#make transectID median FRFy
isTransect = []
betweenNodes = False
transectNum = -1
transectID = []
for x in range(data.shape[0]):
    if data["endPt"][x] and not betweenNodes:
        #first node in time of transect
        betweenNodes = True
        transectNum += 1;
        isTransect.append(True)
        transectID.append(transectNum)
    elif data["endPt"][x] and betweenNodes:
        #last node in time of transect
        betweenNodes = False
        isTransect.append(True)
        transectID.append(transectNum)
    else:
        isTransect.append(betweenNodes)
        if betweenNodes:
            transectID.append(transectNum)
        else:
            transectID.append(-1)
data["isTransect"] = isTransect
data["transectID"] = transectID

In [31]:
transectsOnly = data[data["isTransect"]==True]
plt.scatter(transectsOnly["xFRF"], transectsOnly["yFRF"], c=transectsOnly["transectID"], cmap='hsv', s=1)
cbar=plt.colorbar()
plt.xlabel("FRF Coordinate System X (m)")
plt.ylabel("FRF Coordinate System Y (m)")
cbar.set_label('Transect Number')
plt.title("Transect Isolation Attempt 1")
# plt.savefig("demoTransectIsolationAttempt.png")

Text(0.5, 1.0, 'Transect Isolation Attempt 1')

In [18]:
plt.scatter(data["xFRF"], data["yFRF"], c="black", s=1)
plt.scatter(transectsOnly["xFRF"], transectsOnly["yFRF"], c=transectsOnly["transectID"], cmap='hsv', s=1)
cbar=plt.colorbar()
plt.xlabel("FRF Coordinate System X (m)")
plt.ylabel("FRF Coordinate System Y (m)")
cbar.set_label('Transect Number')
plt.title("Transect Isolation Attempt 1")
plt.savefig("demoTransectIsolationAttemptOverlayed.png")

In [ ]:
data.toExcel